## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
from string import punctuation
from nltk.corpus import stopwords
import os
import re
import emoji
import pandas as pd

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [4]:
# Stopwords
sw = stopwords.words("english")
tw_punct = punctuation - {"#"}

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

def remove_stop(tokens) :
    return([word for word in tokens if word not in sw])

def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    return([t for t in whitespace_pattern.split(text) if t])

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)


In [5]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """
    
    if verbose :        
        print(f"There are {len(tokens)} tokens in the data.")
        print(f"There are {len(set(tokens))} unique tokens in the data.")
        print(f"There are {len(''.join(tokens))} characters in the data.")
        print(f"The lexical diversity is {len(set(tokens))/len(tokens):.3f} in the data.")
    
        counts = Counter(tokens)

        if num_tokens > 0 :
            print(counts.most_common(num_tokens))
        
    return([len(tokens),
           len(set(tokens)),
           len("".join(tokens)),
           len(set(tokens))/len(tokens)])

In [6]:
convention_db = sqlite3.connect('2020_Conventions.db')
convention_cur = convention_db.cursor()

In [12]:
# print tables in convention database
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

convention_cur.execute(sql_query)

In [13]:
print(convention_cur.fetchall())

[('conventions',)]


### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

Let's look at some random entries and see if they look right. 

In [10]:
my_pipeline = [str.lower, remove_punctuation,tokenize]
convention_data = []
query_results = convention_cur.execute(
                            '''
                                SELECT text, party
                                FROM conventions
                                WHERE speaker != "Unknown"
                            ''')
for row in query_results :
    text = ' '.join(prepare(row[0], pipeline=my_pipeline))
    convention_data.append([text, row[1]])

In [11]:
random.choices(convention_data,k=10)

[['this time next year', 'Democratic'],
 ['delaware', 'Republican'],
 ['china would prefer joe biden', 'Republican'],
 ['i said this before if i gave away everything that i have today it would not equal 1 of what i was given when i came to this great country of ours the gift of freedom right now it is up to us to decide our fate and to choose freedom over oppression president trump he’s fighting the forces of anarchy and communism and i know he will continue to do just that what about his opponent and the rest of the dc swamp i have no doubt that we’ll hand the country over to those dangerous forces you and i will decide and here’s what i’ve decided my decision is very easy i choose president trump because i choose america i choose freedom i still hear my dad there is no other place to go thank you and may the good lord bless america',
  'Republican'],
 ['president trump says “this is how we fix it” and i thought “well that’s a simple solution” there’s no other president that could hav

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [14]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2482 as features in the model.


In [15]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    text = set(text.split())
    text = text.intersection(fw)
    
    ret_dict = dict()
    
    for word in text :
        ret_dict[word] = True
    
    return(ret_dict)

In [16]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

AssertionError: 

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [17]:
conv_features("donald is the president", feature_words)

{'is': True, 'donald': True, 'president': True, 'the': True}

In [18]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [19]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [20]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.454


In [21]:
classifier.show_most_informative_features(25)

Most Informative Features
                 radical = True           Republ : Democr =     38.0 : 1.0
                   media = True           Republ : Democr =     35.9 : 1.0
                   votes = True           Democr : Republ =     22.8 : 1.0
             enforcement = True           Republ : Democr =     18.4 : 1.0
                   crime = True           Republ : Democr =     17.2 : 1.0
                 destroy = True           Republ : Democr =     16.2 : 1.0
                freedoms = True           Republ : Democr =     16.2 : 1.0
                   china = True           Republ : Democr =     15.4 : 1.0
                  earned = True           Republ : Democr =     14.1 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0
                  lowest = True           Republ : Democr =     13.0 : 1.0
              prosperity = True           Republ : Democr =     13.0 : 1.0
                   taxes = True           Republ : Democr =     12.8 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

My observation was that the classifier accuracy was lower than expected. 
It's interesting to see how the most informative features are mostly used by Republicans and the only words that were more Democratic dominated were 'votes' and 'climate' (which makes sense to me).



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [22]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [23]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [33]:
tweet_data = []

remove_url = re.compile(r'http\S+')

for candidate, party, text in results :
    text = text.decode("utf-8")
    if text.startswith("b"):
        text = text[1: ]
    text = remove_url.sub("", text)

    tweet_data.append(["".join(prepare(text, pipeline=my_pipeline)), party])

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [34]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [35]:

for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet,feature_words))
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earliertodayispokeonthehousefloorabtprotectinghealthcareforwomenandpraisedppmarmontefortheirworkonthecentralcoast
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: gotribe#rallytogether
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparentlytrumpthinksitsjusttooeasyforstudentsoverwhelmedbythecrushingburdenofdebttopayoffstudentloans#trumpbudget
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: we’regratefulforourfirstrespondersourrescuepersonnelourfirefightersourpoliceandvolunteerswhohavebeenworkingtirelesslytokeeppeoplesafeprovidemuchneededhelpwhileputtingtheirownlivesontheline
Actual party is Republican and our classifer says Democratic.

Here's our (cleaned) tweet: let’smakeitevengreater#kag🇺🇸
Actual party is Republican and our classifer says Democratic.

Here's our (cleaned) tweet: wehaveabout1hruntilthecavstieuptheseries22im#all

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [38]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather than printing them
    tweet_features = conv_features(tweet, feature_words)
   
    # get the estimated party
    estimated_party = classifier.classify(tweet_features)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [39]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 1, 'Democratic': 4371}),
             'Democratic': defaultdict(int,
                         {'Republican': 4, 'Democratic': 5626})})

### Reflections

democratic parties have better chance of being the correct estimated party, though the classifier accuracy is on the low side in general.